In [1]:
import numpy as np
import math
import networkx as nx
import time
from sage.all import *
import matplotlib.pyplot as plt

In [2]:
def random_connected_graph(n, p, seed=None, weighted=True):
    '''
        n - number of vertices
        p - probability there is an edge between two vertices
        uses uniform distribution for edge labeling
    '''
    G = graphs.RandomGNP(n, p, seed=seed) # ensures that G is completely connected
    
    sd = seed
    while len(G.connected_components()) > 1:
        if sd != None:
            sd += 1
        G = graphs.RandomGNP(n, p, seed=sd)
    np.random.seed(seed)
    if weighted:
        for edge in G.edges():
            G.set_edge_label(edge[0], edge[1], RR(np.random.random_sample()))
    return G

def convert_to_complete(G, weighted=False):
    '''
    Function that converts graph to complete graph. Edge weights are distances between the vertices. 
    Weighted as false reduced computational time.
    '''
    all_edges = []
    vertices = G.vertices()
    # distance is symmetric hence iterating one direction is enough
    for i in range(len(vertices)):
        for j in range(i + 1, len(vertices)):
            all_edges.append((vertices[i], vertices[j], G.distance(vertices[i], vertices[j], by_weight=weighted)))
    complete = Graph(all_edges)
    return complete

### Part 1: Traditional p-center computation techniques

Before training our GNN, it is important to develop a framework for generating quality data.  This involves implementing the various algorithms for calculating the k-center.

In [3]:
def k_center(G, k=3, distance=False):
    '''
    Function takes argument graph and modifies into complete graph before implementing efficient k-center algorithm
    Detailed description of algorithm can be found above
    
    '''
    # sorted edge list by edge weight
    complete = convert_to_complete(G, weighted=True)
    weights = sorted(set([edge[2] for edge in complete.edges()]))
    high = len(weights) - 1
    low = 0
    while high - low > 1:
        mid = int(math.ceil((high + low) / 2))
        r_max = weights[mid]
        bottleneck_graph = complete.copy()
        # removes all edges from G that have a weight larger than the maximum permitted radius
        edges_to_remove = [edge for edge in complete.edges() if edge[2] > r_max]
        bottleneck_graph.delete_edges(edges_to_remove)
        centers = bottleneck_graph.dominating_set()
        # binary search within weights
        if len(centers) <= k:
            high = mid
        else:
            low = mid
    if len(centers) > k:
        mid += 1
        r_max = weights[mid]
        bottleneck_graph = complete.copy()
        edges_to_remove = [edge for edge in complete.edges() if edge[2] > r_max]
        bottleneck_graph.delete_edges(edges_to_remove)
        centers = bottleneck_graph.dominating_set()
    if distance:
        return centers, r_max
    else:
        return centers
    
def k_center_approximation(G, k=3, weighted=False, seed=None, distance=False):
    ''' This function uses greedy approximation to find centers.
        Returns the resulting k-centers as specified as well as the maximum distance if required.
    '''
    G.weighted(True)
    vertices = G.vertices()
    if seed != None:
        np.random.seed(seed)
    starting_index = np.random.randint(len(vertices))
    C = [starting_index]
    deltas = []
    while len(C) < k:
        maximized_distance = 0
        for v in range(int(2*len(vertices)/k)):
            if v in C:
                continue
            dists = [float(G.distance(vertices[v], vertices[c], by_weight=weighted)) for c in C]
#             print(dists)
            min_dist = min(dists)
            
            if min_dist > maximized_distance:
                maximized_distance = min_dist
                best_candidate = v
        deltas.append(maximized_distance)
        C.append(best_candidate)
#     C.remove(starting_index)
    if distance:
        return [vertices[c] for c in C], min(deltas)
    else:
        return [vertices[c] for c in C]

In [4]:
n = 100
g = random_connected_graph(n, 0.4, seed=2)
g.weighted(True)

In [5]:
ng = g.networkx_graph(weight_function = lambda edge: float(edge[2]))
pos = nx.spring_layout(ng)
# if weighted:
#     for edge in ng.edges():
#         ng.add_weighted_edges_from([(edge[0],edge[1],l2_norm(pos[edge[0]],pos[edge[1]]))])
# nx.draw(ng, pos, with_labels=True, node_size=30)
# nx.draw(ng, with_labels=True, node_size=3)
# plt.show()

In [6]:
# # GNN imports
!pip3 install --user dgl
!pip3 install --user torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.nn.pytorch import GATConv
from dgl import DGLGraph
import dgl
import networkx as nx
import time
RealNumber = float; Integer = int

distutils: /home/lhj/.local/include/python3.7m/UNKNOWN
sysconfig: /home/lhj/.local/include/python3.7
user = True
home = None
root = None
prefix = None
distutils: /home/lhj/.local/include/python3.7m/UNKNOWN
sysconfig: /home/lhj/.local/include/python3.7
user = True
home = None
root = None
prefix = None


In [7]:
import random
def clusters(G, centers, weighted=True, as_dict=False):    #生成clusts
    if not as_dict:
        clusts = [0] * G.order()
        for vertex in G.vertices():
            if vertex in centers:
                clusts[vertex] = centers.index(vertex)
                continue
            closest = None
            min_dist = 100000000
            # we randomly shuffle to not skew our clustering towards the first elements of centers
            # otherwise, we get a top-heavy list of clusters which, although is correct, will make classification harder
            cs = [*range(len(centers))]
            random.shuffle(cs)
            for c in cs:
                dist = G.distance(centers[c], vertex, by_weight=weighted)
                if dist < min_dist:
                    closest = c
                    min_dist = dist
            clusts[vertex] = closest
        return clusts
    else:
        clusts = {center: [] for center in range(len(centers))}
        for vertex in G.vertices():
            if vertex in centers:
                continue
            closest = None
            min_dist = 100000000
            # we randomly shuffle to not skew our clustering towards the first elements of centers
            # otherwise, we get a top-heavy list of clusters which, although is correct, will make classification harder
            cs = [*range(len(centers))]
            random.shuffle(cs)
            for c in cs:
                dist = G.distance(centers[c], vertex, by_weight=weighted)
                if dist < min_dist:
                    closest = c
                    min_dist = dist
            clusts[closest].append(vertex)
        return clusts

In [8]:
def convert_to_DGLGraph(G, k=3, weighted=True):
    if weighted:
        nxgraph = G.networkx_graph(weight_function = lambda edge: float(edge[2]))
    else:
        nxgraph = G.networkx_graph(weight_function = lambda edge: float(1))
    dgl_G = dgl.DGLGraph()
    dgl_G.from_networkx(nxgraph, edge_attrs=['weight'])
#     centers, distance = k_center(G, k=k, distance=True)
    centers, dist = k_center_approximation(G, k=k, distance=True, weighted=True)
    print(centers)
    print(dist)
    centers = [int(c) for c in centers]
    labels = torch.LongTensor([int(1) if node in centers else int(0) for node in G.vertices()])
    mask = torch.BoolTensor([True for _ in range(len(labels))])
    clusts = clusters(G, centers, weighted=weighted)
    return dgl_G, centers, labels, mask, clusts

def extract_features(G):                #生成features
    vertices = list(G.nodes())
    features = np.zeros((len(vertices), 2))
    for i in vertices:
        features[i] = pos[i]
    return torch.FloatTensor(features)

In [9]:
import dgl.function as fn
#GCN
gcn_msg = fn.copy_u(u='h', out='m')
gcn_reduce = fn.sum(msg='m', out='h')
class GCNLayer(nn.Module):
    def __init__(self, in_feats, out_feats):
        super(GCNLayer, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)

    def forward(self, g, feature):
        # Creating a local scope so that all the stored ndata and edata
        # (such as the `'h'` ndata below) are automatically popped out
        # when the scope exits.
        with g.local_scope():
            g.ndata['h'] = feature
            g.update_all(gcn_msg, gcn_reduce)
            h = g.ndata['h']
            return self.linear(h)

class GCN(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GCN, self).__init__()
        self.layer1 = GCNLayer(in_dim, hidden_dim)
        self.layer2 = GCNLayer(hidden_dim, out_dim)

    def forward(self, g, features):
        x = F.relu(self.layer1(g, features))
        x = self.layer2(g, x)
        return x

In [10]:
t2 = time.time() #计算提取特征的时间
features = extract_features(ng)
tf = time.time() - t2

0.0005502700805664062

In [11]:
k = 3 #计算将graph转化为dgl图的时间
weighted = True
t3 = time.time()
dgl_graph, c, l, m, clsts = convert_to_DGLGraph(g, k=k, weighted=weighted)
td = time.time() - t3

[86, 2, 59]
0.19578382978791675


2.186495304107666

In [12]:
import dgl #训练GCN，输出分类结果
from torch.autograd import Variable
features = Variable(features)
clsts = Variable(torch.LongTensor(clsts))

# net_2_clustering = GAT(in_dim=2,
#       hidden_dim=k*5,
#       out_dim=k,
#       num_heads=5)

# net_2_clustering = GraphSAGE(in_feats=2,
#                           n_hidden=k*5,
#                           n_classes=k,
#                           n_layers=1,
#                           activation=F.relu,
#                           dropout=True,
#                           aggregator_type='mean')
net_2_clustering = GCN(in_dim=2,
                        hidden_dim=k*5,
                        out_dim=k)

optimizer = torch.optim.Adam(net_2_clustering.parameters(), lr=1e-3)

duration = []
losses = []
count = 0

t4 = time.time()

for epoch in range(10000):
    count += 1
    if epoch >= 3:
        t0 = time.time()
        
    net_2_clustering.train()
    logits = net_2_clustering(dgl_graph, features)
    logp = F.log_softmax(logits, 1)
    loss = F.nll_loss(logp, clsts)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        duration.append(time.time() - t0)
        losses.append(loss.item())
    print("Epoch {:05d} | Loss {:.4f} | Time(s) {:.4f}".format(
       count, loss.item(), np.mean(duration)))
#     if epoch%2000 == 0:
#         _, indices = logp.max(dim=1, keepdim=True)
#         cdis = cal_dis(indices)
#         dis = cdis[2]
#         if dis < apxo_solution:
#             break
tt = time.time() - t4
tt

/home/lhj/miniconda3/envs/sage/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/lhj/miniconda3/envs/sage/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00001 | Loss 10.4335 | Time(s) nan
Epoch 00002 | Loss 9.3750 | Time(s) nan
Epoch 00003 | Loss 8.3185 | Time(s) nan
Epoch 00004 | Loss 7.2657 | Time(s) 0.0018
Epoch 00005 | Loss 6.2211 | Time(s) 0.0018
Epoch 00006 | Loss 5.1966 | Time(s) 0.0018
Epoch 00007 | Loss 4.2154 | Time(s) 0.0019
Epoch 00008 | Loss 3.3071 | Time(s) 0.0018
Epoch 00009 | Loss 2.4949 | Time(s) 0.0018
Epoch 00010 | Loss 1.8174 | Time(s) 0.0018
Epoch 00011 | Loss 1.3268 | Time(s) 0.0018
Epoch 00012 | Loss 1.0385 | Time(s) 0.0018
Epoch 00013 | Loss 0.9141 | Time(s) 0.0018
Epoch 00014 | Loss 0.8936 | Time(s) 0.0018
Epoch 00015 | Loss 0.9277 | Time(s) 0.0018
Epoch 00016 | Loss 0.9852 | Time(s) 0.0018
Epoch 00017 | Loss 1.0488 | Time(s) 0.0018
Epoch 00018 | Loss 1.1093 | Time(s) 0.0018
Epoch 00019 | Loss 1.1625 | Time(s) 0.0018
Epoch 00020 | Loss 1.2064 | Time(s) 0.0018
Epoch 00021 | Loss 1.2407 | Time(s) 0.0018
Epoch 00022 | Loss 1.2656 | Time(s) 0.0018
Epoch 00023 | Loss 1.2817 | Time(s) 0.0018
Epoch 00024 | Loss 

Epoch 00208 | Loss 0.5331 | Time(s) 0.0018
Epoch 00209 | Loss 0.5328 | Time(s) 0.0018
Epoch 00210 | Loss 0.5325 | Time(s) 0.0018
Epoch 00211 | Loss 0.5322 | Time(s) 0.0018
Epoch 00212 | Loss 0.5318 | Time(s) 0.0018
Epoch 00213 | Loss 0.5315 | Time(s) 0.0018
Epoch 00214 | Loss 0.5312 | Time(s) 0.0018
Epoch 00215 | Loss 0.5309 | Time(s) 0.0018
Epoch 00216 | Loss 0.5306 | Time(s) 0.0018
Epoch 00217 | Loss 0.5303 | Time(s) 0.0018
Epoch 00218 | Loss 0.5299 | Time(s) 0.0018
Epoch 00219 | Loss 0.5296 | Time(s) 0.0018
Epoch 00220 | Loss 0.5293 | Time(s) 0.0018
Epoch 00221 | Loss 0.5290 | Time(s) 0.0018
Epoch 00222 | Loss 0.5287 | Time(s) 0.0018
Epoch 00223 | Loss 0.5284 | Time(s) 0.0018
Epoch 00224 | Loss 0.5281 | Time(s) 0.0018
Epoch 00225 | Loss 0.5278 | Time(s) 0.0018
Epoch 00226 | Loss 0.5275 | Time(s) 0.0018
Epoch 00227 | Loss 0.5272 | Time(s) 0.0018
Epoch 00228 | Loss 0.5269 | Time(s) 0.0018
Epoch 00229 | Loss 0.5266 | Time(s) 0.0018
Epoch 00230 | Loss 0.5264 | Time(s) 0.0018
Epoch 00231

Epoch 00415 | Loss 0.4994 | Time(s) 0.0018
Epoch 00416 | Loss 0.4993 | Time(s) 0.0018
Epoch 00417 | Loss 0.4993 | Time(s) 0.0018
Epoch 00418 | Loss 0.4992 | Time(s) 0.0018
Epoch 00419 | Loss 0.4991 | Time(s) 0.0018
Epoch 00420 | Loss 0.4991 | Time(s) 0.0018
Epoch 00421 | Loss 0.4990 | Time(s) 0.0018
Epoch 00422 | Loss 0.4989 | Time(s) 0.0018
Epoch 00423 | Loss 0.4988 | Time(s) 0.0018
Epoch 00424 | Loss 0.4988 | Time(s) 0.0018
Epoch 00425 | Loss 0.4987 | Time(s) 0.0018
Epoch 00426 | Loss 0.4986 | Time(s) 0.0018
Epoch 00427 | Loss 0.4986 | Time(s) 0.0018
Epoch 00428 | Loss 0.4985 | Time(s) 0.0018
Epoch 00429 | Loss 0.4984 | Time(s) 0.0018
Epoch 00430 | Loss 0.4983 | Time(s) 0.0018
Epoch 00431 | Loss 0.4983 | Time(s) 0.0018
Epoch 00432 | Loss 0.4982 | Time(s) 0.0018
Epoch 00433 | Loss 0.4981 | Time(s) 0.0018
Epoch 00434 | Loss 0.4981 | Time(s) 0.0018
Epoch 00435 | Loss 0.4980 | Time(s) 0.0018
Epoch 00436 | Loss 0.4979 | Time(s) 0.0018
Epoch 00437 | Loss 0.4979 | Time(s) 0.0018
Epoch 00438

Epoch 00621 | Loss 0.4870 | Time(s) 0.0019
Epoch 00622 | Loss 0.4869 | Time(s) 0.0019
Epoch 00623 | Loss 0.4869 | Time(s) 0.0019
Epoch 00624 | Loss 0.4868 | Time(s) 0.0019
Epoch 00625 | Loss 0.4868 | Time(s) 0.0019
Epoch 00626 | Loss 0.4867 | Time(s) 0.0019
Epoch 00627 | Loss 0.4867 | Time(s) 0.0019
Epoch 00628 | Loss 0.4867 | Time(s) 0.0019
Epoch 00629 | Loss 0.4866 | Time(s) 0.0019
Epoch 00630 | Loss 0.4866 | Time(s) 0.0019
Epoch 00631 | Loss 0.4865 | Time(s) 0.0019
Epoch 00632 | Loss 0.4865 | Time(s) 0.0019
Epoch 00633 | Loss 0.4864 | Time(s) 0.0019
Epoch 00634 | Loss 0.4864 | Time(s) 0.0019
Epoch 00635 | Loss 0.4863 | Time(s) 0.0019
Epoch 00636 | Loss 0.4863 | Time(s) 0.0019
Epoch 00637 | Loss 0.4863 | Time(s) 0.0019
Epoch 00638 | Loss 0.4862 | Time(s) 0.0019
Epoch 00639 | Loss 0.4862 | Time(s) 0.0019
Epoch 00640 | Loss 0.4861 | Time(s) 0.0019
Epoch 00641 | Loss 0.4861 | Time(s) 0.0019
Epoch 00642 | Loss 0.4860 | Time(s) 0.0019
Epoch 00643 | Loss 0.4860 | Time(s) 0.0019
Epoch 00644

...

Epoch 08991 | Loss 0.2056 | Time(s) 0.0019
Epoch 08992 | Loss 0.2056 | Time(s) 0.0019
Epoch 08993 | Loss 0.2056 | Time(s) 0.0019
Epoch 08994 | Loss 0.2056 | Time(s) 0.0019
Epoch 08995 | Loss 0.2055 | Time(s) 0.0019
Epoch 08996 | Loss 0.2055 | Time(s) 0.0019
Epoch 08997 | Loss 0.2055 | Time(s) 0.0019
Epoch 08998 | Loss 0.2055 | Time(s) 0.0019
Epoch 08999 | Loss 0.2055 | Time(s) 0.0019
Epoch 09000 | Loss 0.2054 | Time(s) 0.0019
Epoch 09001 | Loss 0.2054 | Time(s) 0.0019
Epoch 09002 | Loss 0.2054 | Time(s) 0.0019
Epoch 09003 | Loss 0.2054 | Time(s) 0.0019
Epoch 09004 | Loss 0.2054 | Time(s) 0.0019
Epoch 09005 | Loss 0.2053 | Time(s) 0.0019
Epoch 09006 | Loss 0.2053 | Time(s) 0.0019
Epoch 09007 | Loss 0.2053 | Time(s) 0.0019
Epoch 09008 | Loss 0.2053 | Time(s) 0.0019
Epoch 09009 | Loss 0.2053 | Time(s) 0.0019
Epoch 09010 | Loss 0.2053 | Time(s) 0.0019
Epoch 09011 | Loss 0.2053 | Time(s) 0.0019
Epoch 09012 | Loss 0.2053 | Time(s) 0.0019
Epoch 09013 | Loss 0.2053 | Time(s) 0.0019
Epoch 09014

Epoch 09249 | Loss 0.2033 | Time(s) 0.0019
Epoch 09250 | Loss 0.2033 | Time(s) 0.0019
Epoch 09251 | Loss 0.2034 | Time(s) 0.0019
Epoch 09252 | Loss 0.2034 | Time(s) 0.0019
Epoch 09253 | Loss 0.2034 | Time(s) 0.0019
Epoch 09254 | Loss 0.2034 | Time(s) 0.0019
Epoch 09255 | Loss 0.2033 | Time(s) 0.0019
Epoch 09256 | Loss 0.2033 | Time(s) 0.0019
Epoch 09257 | Loss 0.2032 | Time(s) 0.0019
Epoch 09258 | Loss 0.2032 | Time(s) 0.0019
Epoch 09259 | Loss 0.2031 | Time(s) 0.0019
Epoch 09260 | Loss 0.2031 | Time(s) 0.0019
Epoch 09261 | Loss 0.2031 | Time(s) 0.0019
Epoch 09262 | Loss 0.2031 | Time(s) 0.0019
Epoch 09263 | Loss 0.2031 | Time(s) 0.0019
Epoch 09264 | Loss 0.2031 | Time(s) 0.0019
Epoch 09265 | Loss 0.2031 | Time(s) 0.0019
Epoch 09266 | Loss 0.2031 | Time(s) 0.0019
Epoch 09267 | Loss 0.2031 | Time(s) 0.0019
Epoch 09268 | Loss 0.2031 | Time(s) 0.0019
Epoch 09269 | Loss 0.2031 | Time(s) 0.0019
Epoch 09270 | Loss 0.2031 | Time(s) 0.0019
Epoch 09271 | Loss 0.2031 | Time(s) 0.0019
Epoch 09272

Epoch 09512 | Loss 0.2012 | Time(s) 0.0019
Epoch 09513 | Loss 0.2012 | Time(s) 0.0019
Epoch 09514 | Loss 0.2012 | Time(s) 0.0019
Epoch 09515 | Loss 0.2011 | Time(s) 0.0019
Epoch 09516 | Loss 0.2011 | Time(s) 0.0019
Epoch 09517 | Loss 0.2011 | Time(s) 0.0019
Epoch 09518 | Loss 0.2011 | Time(s) 0.0019
Epoch 09519 | Loss 0.2011 | Time(s) 0.0019
Epoch 09520 | Loss 0.2011 | Time(s) 0.0019
Epoch 09521 | Loss 0.2011 | Time(s) 0.0019
Epoch 09522 | Loss 0.2010 | Time(s) 0.0019
Epoch 09523 | Loss 0.2010 | Time(s) 0.0019
Epoch 09524 | Loss 0.2010 | Time(s) 0.0019
Epoch 09525 | Loss 0.2010 | Time(s) 0.0019
Epoch 09526 | Loss 0.2010 | Time(s) 0.0019
Epoch 09527 | Loss 0.2010 | Time(s) 0.0019
Epoch 09528 | Loss 0.2010 | Time(s) 0.0019
Epoch 09529 | Loss 0.2010 | Time(s) 0.0019
Epoch 09530 | Loss 0.2010 | Time(s) 0.0019
Epoch 09531 | Loss 0.2010 | Time(s) 0.0019
Epoch 09532 | Loss 0.2010 | Time(s) 0.0019
Epoch 09533 | Loss 0.2010 | Time(s) 0.0019
Epoch 09534 | Loss 0.2010 | Time(s) 0.0019
Epoch 09535

Epoch 09768 | Loss 0.1993 | Time(s) 0.0019
Epoch 09769 | Loss 0.1993 | Time(s) 0.0019
Epoch 09770 | Loss 0.1993 | Time(s) 0.0019
Epoch 09771 | Loss 0.1993 | Time(s) 0.0019
Epoch 09772 | Loss 0.1993 | Time(s) 0.0019
Epoch 09773 | Loss 0.1993 | Time(s) 0.0019
Epoch 09774 | Loss 0.1993 | Time(s) 0.0019
Epoch 09775 | Loss 0.1993 | Time(s) 0.0019
Epoch 09776 | Loss 0.1993 | Time(s) 0.0019
Epoch 09777 | Loss 0.1993 | Time(s) 0.0019
Epoch 09778 | Loss 0.1993 | Time(s) 0.0019
Epoch 09779 | Loss 0.1993 | Time(s) 0.0019
Epoch 09780 | Loss 0.1993 | Time(s) 0.0019
Epoch 09781 | Loss 0.1992 | Time(s) 0.0019
Epoch 09782 | Loss 0.1992 | Time(s) 0.0019
Epoch 09783 | Loss 0.1992 | Time(s) 0.0019
Epoch 09784 | Loss 0.1992 | Time(s) 0.0019
Epoch 09785 | Loss 0.1992 | Time(s) 0.0019
Epoch 09786 | Loss 0.1992 | Time(s) 0.0019
Epoch 09787 | Loss 0.1992 | Time(s) 0.0019
Epoch 09788 | Loss 0.1992 | Time(s) 0.0019
Epoch 09789 | Loss 0.1992 | Time(s) 0.0019
Epoch 09790 | Loss 0.1992 | Time(s) 0.0019
Epoch 09791

22.285849571228027

In [13]:
_, indices = logp.max(dim=1, keepdim=True) #输出分类的结果
indices = indices.reshape(n)
indices = indices.detach().numpy()
indices = indices.tolist()
def get_same_element_index(ob_list, word):
    return [i for (i, v) in enumerate(ob_list) if v == word]
v = []
for i in range(k):
    a = get_same_element_index(indices, i)
    v.append(a)

In [14]:
G = []
for i in range(k):
    G.append(g.subgraph(v[i]))
center = []
for j in range(k):
    c = G[j].center(by_weight=True)
    center.append(c[0])
center

[85, 2, 16]

In [15]:
t7=time.time()
dist=[]
for i in range(k):
    dists = []
    for j in v[i]:
        dists.append(g.distance(center[i], j, by_weight=True))
    dist.append(np.max(dists))
minmax_dist = np.max(dist)
tm = time.time()-t2
print("GCN solution")
print('Centers:',center)
print('Maximum distance to any point:' + str(minmax_dist)) ; print('\n')
print('runtime: %d', tm)

GCN solution
Centers: [85, 2, 16]
Maximum distance to any point:0.18577072103325332


runtime: %d 25.018150091171265


In [16]:
t1 = time.time()
centers, dist = k_center_approximation(g, k=3, distance=True, weighted=True)
ta = time.time() - t1
p_approximate = g.plot(layout='circular', vertex_colors={'red': centers}, vertex_labels=False, edge_labels=True)

print("K-Center approximation")
print('Centers:',centers)
print('Maximum distance to any point:' + str(dist)) ; print('\n')
print('runtime: %d', ta)
p_approximate.show(figsize=15)

t0 = time.time()
centers, dist = k_center(g, k=3, distance=True)
te = time.time() - t0
p_exact = g.plot(layout='circular', vertex_colors={'red': centers}, vertex_labels=False, edge_labels=True)

print("Exact solution")
print('Centers:',centers)
print('Maximum distance to any point:' + str(dist)) ; print('\n')
print('runtime: %d', te)
p_exact.show(figsize=15)


K-Center approximation
Centers: [27, 59, 18]
Maximum distance to any point:0.2548802394193491


runtime: %d 0.9802792072296143
Graphics object consisting of 3899 graphics primitives
Exact solution
Centers: [9, 67, 97]
Maximum distance to any point:0.1424666046560129


runtime: %d 22.574922561645508
Graphics object consisting of 3899 graphics primitives
